# CAPO_TCS Data Cleaning

### 1) Load the raw dataset

In [3]:
import pandas as pd

df_raw = pd.read_csv("capo_data.csv")
print("rows, columns:", df_raw.shape)
df_raw.head()

rows, columns: (8230, 100)


,case_id,dem_age,dem_sex,dem_pregnant,dem_trimester,exam_height,exam_weight,exam_hr,exam_rr,exam_sbp,...,day6test___wbc,day6test___oral,day7test___cough,day7test___afebrile,day7test___wbc,day7test___oral,over7daytest___cough,over7daytest___afebrile,over7daytest___wbc,over7daytest___oral
0,16,43.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,1,1,1,1
1,17,60.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,18,45.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,0,0,0,0,0,0,0,0
3,19,35.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,1,1,1,1,1
4,20,61.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


### 2) Confirm the daily stability criteria columns

In [5]:
# show only the columns that contain "day" and "test"
day_cols = [c for c in df_raw.columns if "day" in c and "test" in c]

print("number of day-test columns:", len(day_cols))
print("first 30 day-test columns:")
print(day_cols[:30])

number of day-test columns: 36
first 30 day-test columns:
['day0test___cough', 'day0test___afebrile', 'day0test___wbc', 'day0test___oral', 'day1test___cough', 'day1test___afebrile', 'day1test___wbc', 'day1test___oral', 'day2test___cough', 'day2test___afebrile', 'day2test___wbc', 'day2test___oral', 'day3test___cough', 'day3test___afebrile', 'day3test___wbc', 'day3test___oral', 'day4test___cough', 'day4test___afebrile', 'day4test___wbc', 'day4test___oral', 'day5test___cough', 'day5test___afebrile', 'day5test___wbc', 'day5test___oral', 'day6test___cough', 'day6test___afebrile', 'day6test___wbc', 'day6test___oral', 'day7test___cough', 'day7test___afebrile']


### 3) Compute Time to Clinical Stability (TCS)
##### The patient's TCS is the first day where all four criteria equal 1. 
##### If the patient does not meet all four criteria by day 7, we set TCS = 8.

In [7]:
df = df_raw.copy()

criteria = ["cough", "afebrile", "wbc", "oral"]

# default everyone to TCS = 8 (not stable by day 7)
df["TCS"] = 8

for day in range(1, 8):
    
    cols = [f"day{day}test___{c}" for c in criteria]
    
    stable_today = (
        (df[cols[0]] == 1) &
        (df[cols[1]] == 1) &
        (df[cols[2]] == 1) &
        (df[cols[3]] == 1)
    )
    
    df.loc[stable_today & (df["TCS"] == 8), "TCS"] = day

df["TCS"].value_counts().sort_index()

TCS
1     669
2    1135
3    1151
4     990
5     695
6     397
7     349
8    2844
Name: count, dtype: int64

### 4) Create binary targets for logistic regression
##### stable_7: 1 if TCS <= 7 else 0 (not stable by day 7)
##### early_tcs: 1 if TCS < 4 else 0 (early vs late stability)

In [9]:
df["stable_7"] = (df["TCS"] <= 7).astype(int)
df["early_tcs"] = (df["TCS"] < 4).astype(int)

print("stable by day 7 rate:")
print(df["stable_7"].mean())

print("\nearly stability rate:")
print(df["early_tcs"].mean())

stable by day 7 rate:
0.6544349939246659

early stability rate:
0.35905224787363305


### 5) Assess missing data (columns)

In [11]:
missing_percent = df.isna().mean() * 100
missing_percent = missing_percent.sort_values(ascending=False)

print(missing_percent.head(20))

lab_vitamind         99.939247
lab_hga1c            99.914945
dem_trimester        99.769137
lab_lactate          99.088700
lab_inr              98.359660
lab_ckmb3            98.311057
lab_bands            98.238153
lab_bilirubin        97.339004
lab_ckmb2            96.318348
lab_trop2            95.018226
cx_du                95.018226
cx_db                94.835966
lab_pct              93.693803
lab_bnp              93.074119
lab_ast              92.758202
lab_alt              92.369380
lab_ldl              92.272175
lab_hdl              90.741191
lab_ckmb1            89.441069
lab_triglycerides    89.380316
dtype: float64


### 6) Drop variables > 90% missing

In [13]:
high_missing = missing_percent[missing_percent > 90]

print("columns > 90% missing:", len(high_missing))
print(high_missing)

columns > 90% missing: 18
lab_vitamind     99.939247
lab_hga1c        99.914945
dem_trimester    99.769137
lab_lactate      99.088700
lab_inr          98.359660
lab_ckmb3        98.311057
lab_bands        98.238153
lab_bilirubin    97.339004
lab_ckmb2        96.318348
lab_trop2        95.018226
cx_du            95.018226
cx_db            94.835966
lab_pct          93.693803
lab_bnp          93.074119
lab_ast          92.758202
lab_alt          92.369380
lab_ldl          92.272175
lab_hdl          90.741191
dtype: float64


In [15]:
cols_to_drop = high_missing.index.tolist()

df = df.drop(columns=cols_to_drop)

print("new shape after drop:", df.shape)

new shape after drop: (8230, 85)


### 7) Re-check missingness after dropping sparse columns

In [17]:
missing_percent = df.isna().mean() * 100
missing_percent = missing_percent.sort_values(ascending=False)

print(missing_percent.head(20))

lab_ckmb1            89.441069
lab_triglycerides    89.380316
dem_pregnant         89.137303
lab_cholesterol      88.639125
lab_trop3            87.205346
lab_trop1            86.257594
lab_ldh              84.532199
lab_abgfio2          79.939247
lab_abgph            79.392467
exam_fio2            77.618469
lab_albumin          75.759417
lab_abgpao2          73.633050
lab_bicarb           72.818955
lab_abgbicarb        70.984204
lab_crp              70.874848
lab_abgpaco2         70.486027
lab_bun              64.301337
exam_rr              62.345079
exam_height          60.959903
lab_glucose          60.838396
dtype: float64


### 8) Impute remaining missing values
##### Numeric columns --> median
##### Categorical columns --> mode

In [19]:
from sklearn.impute import SimpleImputer

df_impute = df.copy()

num_cols = df_impute.select_dtypes(include=["float64","int64"]).columns
cat_cols = df_impute.select_dtypes(exclude=["float64","int64"]).columns

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

df_impute[num_cols] = num_imputer.fit_transform(df_impute[num_cols])

if len(cat_cols) > 0:
    df_impute[cat_cols] = cat_imputer.fit_transform(df_impute[cat_cols])

print("remaining missing values:", df_impute.isna().sum().sum())

remaining missing values: 0


### 9) Plausibility check
##### Review summary statistics for certain variables (height/weight) to identify obviously invalid values.

In [21]:
print(df_impute["exam_height"].describe())
print(df_impute["exam_weight"].describe())

count    8230.000000
mean      166.127947
std        21.793088
min        15.000000
25%       169.000000
50%       169.000000
75%       169.000000
max       223.000000
Name: exam_height, dtype: float64
count    8230.000000
mean       71.398906
std        13.107330
min         0.000000
25%        70.000000
50%        70.000000
75%        70.000000
max       320.000000
Name: exam_weight, dtype: float64


### 10) Correct invalis values
##### Weight = 0 --> median weight
##### Height < 100 cm --> median height

In [23]:
df_impute.loc[df_impute["exam_weight"] == 0, "exam_weight"] = df_impute["exam_weight"].median()

df_impute.loc[df_impute["exam_height"] < 100, "exam_height"] = df_impute["exam_height"].median()

### 11) Extra validation checks
##### row missingness distribution: confirms rows are not overwhelmingly empty
##### low-variance columns: confirms no constant/near-constant predictors

In [29]:
# extra validation checks

# row missingness distribution
row_missing_pct = df.isna().mean(axis=1) * 100
print("row missingness distribution:")
print(row_missing_pct.describe())

# low variance column check
low_var_cols = [c for c in df.columns if df[c].nunique() <= 1]
print("\nlow variance columns:")
print(low_var_cols)

row missingness distribution:
count    8230.000000
mean       27.576442
std        14.182167
min         0.000000
25%        12.941176
50%        37.647059
75%        41.176471
max        52.941176
dtype: float64

low variance columns:
[]


### 12) Export cleaned dataset

In [31]:
df_impute.to_csv("capo_tcs_data_cleaned.csv", index=False)